In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.selenium_manager import SeleniumManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from googletrans import Translator
from google_trans_new import google_translator  
from deep_translator import GoogleTranslator
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
import warnings
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType
import pyspark.pandas as ps
warnings.filterwarnings('ignore')

c:\Users\ayush\anaconda3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
	print(key,value)
	dropdown = driver.find_element(by=By.ID,value=key)
	dropdown_select = Select(dropdown)
	for option in dropdown_select.options:
		if option.text == value:
			option.click()
			print(f'Clicked {option.text} for {key}')
			break
	driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
captcha = input()
driver.find_element(by=By.ID, value='cpatchaTextBox').send_keys(captcha)
print(f'Entered Captcha {captcha}')
driver.implicitly_wait(2)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
driver.implicitly_wait(10)


#Click on 50 Pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
	if option.text == 'All':
		option.click()
		print('Set to All Records')
		break

print('So Far, So Good')

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Entered Captcha KUOJRP
Set to All Records
So Far, So Good


In [4]:
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

for index, table in enumerate(driver.find_elements(by=By.ID, value='tbdata')):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    records_raw.loc[len(records_raw)] = data
    # print(data)
    print(f'{index} Row appended')

records_raw
records_translate = records_raw.copy()

0 Row appended
1 Row appended
2 Row appended
3 Row appended
4 Row appended
5 Row appended
6 Row appended
7 Row appended
8 Row appended
9 Row appended
10 Row appended
11 Row appended
12 Row appended
13 Row appended
14 Row appended
15 Row appended
16 Row appended
17 Row appended
18 Row appended
19 Row appended
20 Row appended
21 Row appended
22 Row appended
23 Row appended
24 Row appended
25 Row appended
26 Row appended
27 Row appended
28 Row appended
29 Row appended
30 Row appended
31 Row appended
32 Row appended
33 Row appended
34 Row appended
35 Row appended
36 Row appended
37 Row appended
38 Row appended
39 Row appended
40 Row appended
41 Row appended
42 Row appended
43 Row appended
44 Row appended
45 Row appended
46 Row appended
47 Row appended
48 Row appended
49 Row appended
50 Row appended
51 Row appended
52 Row appended
53 Row appended
54 Row appended
55 Row appended
56 Row appended
57 Row appended
58 Row appended
59 Row appended
60 Row appended
61 Row appended
62 Row appended
63

In [6]:
# PostgreSQL database connection parameters
db_user = 'postgres'
db_password = 'Sunrise12345'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propReturns'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
table_name = 'record_details_raw'

records_raw.to_sql(table_name, engine, if_exists='replace', index=True)

print(f"{len(records_raw)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_raw' table.


In [8]:
# #Approach Via Pandas --> Extremely Slow
# records_translate = records_raw.copy()
# records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
# cols_to_translate = [
#     'diarrhea type',
#     'Du. Prohibit. Office',
#     'Will write', 
#     'Will write down', 
#     'Other information'
#     ]

# records_translated = pd.DataFrame(columns=records_translate.columns)
# batch_size = 100
# start = 0
# for batch in range(start,len(records_translate),batch_size):
#     print(f'Batch -> [{batch},{batch+batch_size}]')
#     temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
#     for col in cols_to_translate:
#         print(f'Translating {col}')
#         temp_df[col] = temp_df[col].apply(lambda x: translator(x,source='hi'))
#         print(f'Done')
#     records_translated = pd.concat([records_translated,temp_df])
# records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

Batch -> [0,100]
Translating diarrhea type
Done
Translating Du. Prohibit. Office
Done
Translating Will write


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again

In [40]:
# Approach Via Spark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['spark_home'] = sys.executable
os.environ['hadoop_home'] = sys.executable
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark

In [42]:
# del spark_df 
spark_df = spark.createDataFrame(records_raw.copy())

In [43]:
new_names = []
for col in records_raw.columns:
    new_names.append(translator(col,source='auto'))

new_names = [x.replace('.','') for x in new_names]
spark_df = spark_df.toDF(*new_names)

In [45]:
from googletrans import Translator
def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

translate_udf = udf(translator_v2,StringType())

In [48]:
cols_to_translate = [
    'diarrhea type',
    'Du Prohibit Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

for column in cols_to_translate:
    spark_df = spark_df.withColumn(column, translate_udf(spark_df[column]))

diarrhea type
Du Prohibit Office
Will write
Will write down
Other information


In [49]:
spark_df = spark_df.withColumnRenamed(existing='Will write',new='buyer_name')
spark_df = spark_df.withColumnRenamed(existing='Will write down',new='seller_name')

In [53]:
records_translated = spark_df.toPandas()

In [55]:
records_translated = spark_df.toPandas()
table_name = 'record_details_translated'
records_translated.to_sql(table_name, engine, if_exists='replace', index=True)
print(f"{len(records_translate)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_translated' table.
